# 複素ANNによる非線形歪補償
複素数を入力とする3層ANNによる補償

In [120]:
#import
import sys
import os
import time
import datetime
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms

sys.path.append('../')
from pyopt.util import save_pickle, load_pickle

# 1. Preprocessing

## 1.1 データの整形

In [19]:
def data_shaping(input_signal, signal, max_tap, tap):
    """
    input_signal: 伝送前の信号
    signal: 伝送後の信号
    max_tap: 最大の同時入力シンボル数
    tap: 同時入力シンボル数
    
    signal = [x_0, x_1, ... , x_(n-1)]
      |
      |
      v
    x = [[x_0, x_1, ... , x_tap-1],
            [x_1, x_2, ..., x_tap],
                   .
                   .
                   .
            [x_(n-tap), x_(n-tap+1), ..., x(n-1)]]
      |
      |
      v
    x = [[i_0, q_0, i_1, q_1, ... , i_(tap-1), q_(tap-1)],
            [i_1, q_1, i_2, q_2, ... , i_tap, q_tap],
                   .
                   .
                   .
            [i_(n-tap), q_(n-tap), i_(n-tap+1), q_(n-tap+1), ..., i_(n-1), q_(n-1)]] (batch, input_dim) input_dim = tap * 2
    
    y  (batch, output_dim) output_dim = 2
    """
    
    x = np.zeros((len(input_signal) - (max_tap - 1), tap, 2), dtype=float)
    y = np.zeros((len(input_signal) - (max_tap - 1), 2), dtype=float)
    for i, j in enumerate(np.arange(max_tap // 2, len(input_signal) - max_tap // 2)):
        x[i, :, 0] = signal[j - tap // 2: j + tap // 2 + 1].real
        x[i, :, 1] = signal[j - tap // 2: j + tap // 2 + 1].imag
        y[i, 0] = input_signal[j].real
        y[i, 1] = input_signal[j].imag
    return x, y

In [106]:
#動作確認
tap = 29
max_tap = 51

df_dir = '../data/input/prbs.csv'
df = pd.read_csv(df_dir, index_col=0)  # dataframe読み込み
condition = (df['N']==13) & (df['itr']==1) & (df['form']=='RZ16QAM') & (df['n']==32) & (df['equalize']==False) & (df['baudrate']==28) & (df['PdBm']==1)
sgnl = load_pickle(df[condition].iloc[0]['data_path'])  # dataframeから条件と合う行を取得し,pickleの保存先(data_path)にアクセス
lc = sgnl.linear_compensation(500, sgnl.signal['x_500'])
x, y = data_shaping(sgnl.signal['x_0'][16::32], lc[16::32], max_tap, tap)  # ANNに入力できるようにデータを整形

print('x size: ', x.shape)
print('y size: ', y.shape)
print(x[0])
print(y[0])

x size:  (1998, 29, 2)
y size:  (1998, 2)
[[ 51598.75641186 -53403.3657517 ]
 [-32145.84759244  96508.43591778]
 [-30606.37446336  95283.58932182]
 [ 32503.60952844  13910.52002391]
 [  8248.62335619  73391.81300168]
 [ 29107.6877347   14808.73597731]
 [-78280.98856905  15366.79259349]
 [-76605.44555413  12791.23534176]
 [ 69345.95709822  -8729.55171237]
 [ 36932.5922594   11156.38959978]
 [ 13056.01107017 -32846.73484666]
 [ 10610.89876389 -34963.38298674]
 [-73494.72750711   7190.0287674 ]
 [-49860.52436413  54128.52098679]
 [-53019.95429361  53670.26520552]
 [ 32274.63575034 -91081.39804461]
 [-14385.94066463  30341.90659466]
 [-47630.74129796  49235.95785073]
 [-11546.66807361 -75020.35362366]
 [-11274.86250656  31430.37354787]
 [-51761.20934421  52640.09062926]
 [-92079.7374958  -28400.06490156]
 [-54174.37271402  50576.41182636]
 [-67054.54768817  15313.41343857]
 [ 96034.7954907   33242.87671139]
 [ 50154.90489795 -49671.1420593 ]
 [-93925.54099544 -34256.15458994]
 [-30446.5662

In [29]:
x_tensor = torch.Tensor(x)
x_complex = torch.view_as_complex(x_tensor)
print(x_complex)

tensor([[ 51598.7578-53403.3672j, -32145.8477+96508.4375j,
         -30606.3750+95283.5859j,  ...,
         -93925.5391-34256.1562j, -30446.5664-13048.7129j,
           9693.4248-29001.5801j],
        [-32145.8477+96508.4375j, -30606.3750+95283.5859j,
          32503.6094+13910.5205j,  ...,
         -30446.5664-13048.7129j,   9693.4248-29001.5801j,
          53869.6953+53203.7969j],
        [-30606.3750+95283.5859j,  32503.6094+13910.5205j,
           8248.6230+73391.8125j,  ...,
           9693.4248-29001.5801j,  53869.6953+53203.7969j,
         -74834.9922+12591.7324j],
        ...,
        [-57464.2109-54496.4727j, -71785.1406+16135.0635j,
         -37748.6445-11243.9277j,  ...,
         -10821.5039-70501.7188j, -72369.8047+16814.8672j,
         -10232.1523+33094.5391j],
        [-71785.1406+16135.0635j, -37748.6445-11243.9277j,
          96017.2891+27587.5176j,  ...,
         -72369.8047+16814.8672j, -10232.1523+33094.5391j,
         -12742.2695-76211.6172j],
        [-37748.6445-1

## 1.2 平均,標準偏差の計算

In [25]:
mean = np.mean(x)
std = np.std(x)

print('mean: ', mean)
print('std: ', std)

mean:  1153.2726078905946
std:  52094.955682540436


# 2. Dataset定義

In [114]:
class Dataset(data.Dataset):
    def __init__(self, x, y, mean, std):
        self.x, self.y, self.mean, self.std = x, y, mean, std
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]
        
        x = (x - self.mean) / self.std
        y = (y - self.mean) / self.std
        
        x = torch.Tensor(x)
        y = torch.Tensor(y)
        
        x_i = x[:, 0]
        x_q = x[:, 1]
        y_i = y[0]
        y_q = y[1]
        return x_i, x_q, y_i, y_q

In [115]:
#動作確認
train_dataset = Dataset(x=x, y=y, mean=mean, std=std)

index = 0
x_i, x_q, y_i, y_q = train_dataset.__getitem__(index)
x_array = x_i.detach().numpy()

print('mean: ', np.mean(x_array))
print('std: ', np.std(x_array))
print(x_i)
print(y_i)

mean:  -0.3059248
std:  0.9654936
tensor([ 0.9683, -0.6392, -0.6096,  0.6018,  0.1362,  0.5366, -1.5248, -1.4926,
         1.3090,  0.6868,  0.2285,  0.1815, -1.4329, -0.9792, -1.0399,  0.5974,
        -0.2983, -0.9364, -0.2438, -0.2386, -1.0157, -1.7897, -1.0621, -1.3093,
         1.8213,  0.9406, -1.8251, -0.6066,  0.1639])
tensor(-1.3750)


In [116]:
batch_size = 100

train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

dataloaders_dict = {'train': train_dataloader}

# 3. Model定義

In [136]:
def zrelu(x_i, x_q):
    x_i, x_q = x_i.clone(), x_q.clone()
    for i in range(x_i.shape[0]):
        for j in range(x_i.shape[1]):
            if x_i[i, j] < 0 or x_q[i, j] < 0:
                x_i[i, j] = 0
                x_q[i, j] = 0
    return x_i, x_q

In [99]:
def crelu(x_i, x_q):
    x_i, x_q = x_i.clone(), x_q.clone()
    x_i, x_q = F.relu(x_i), F.relu(x_q)
    return x_i, x_q

In [123]:
class ComplexLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        # 重み定義 Xavierの初期値
        k = 1 / in_features
        weight_i = torch.empty(out_features, in_features).uniform_(-math.sqrt(k), math.sqrt(k))
        self.weight_i = nn.Parameter(weight_i)
        weight_q = torch.empty(out_features, in_features).uniform_(-math.sqrt(k), math.sqrt(k))
        self.weight_q = nn.Parameter(weight_q)
        
        bias_i = torch.empty(out_features).uniform_(-k, k)
        self.bias_i = nn.Parameter(bias_i)
        bias_q = torch.empty(out_features).uniform_(-k, k)
        self.bias_q = nn.Parameter(bias_q)
        
    def forward(self, x_i, x_q):
        i = nn.functional.linear(x_i, self.weight_i, self.bias_i)
        q = nn.functional.linear(x_q, self.weight_q, self.bias_q)
        return i - q, i + q

In [118]:
class ComplexANN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_neuron):
        super().__init__()
        self.fc1 = ComplexLinear(input_dim, hidden_neuron)
        self.fc2 = ComplexLinear(hidden_neuron, output_dim)
    
    def forward(self, x_i, x_q):
        x_i, x_q = self.fc1(x_i, x_q)
        x_i, x_q = zrelu(x_i, x_q)
        x_i, x_q = self.fc2(x_i, x_q)
        return x_i, x_q

In [139]:
#動作確認
hidden_neuron = 300
device = torch.device('cpu') # 'cuda' if torch.cuda.is_available() else 
print('Device available now:', device)

model = ComplexANN(input_dim=tap, output_dim=1, hidden_neuron=hidden_neuron).to(device)
for x_i, x_q, y_i, y_q in train_dataloader:
    out_i, out_q = model(x_i, x_q)
    print(output)
    print(out_i.shape)
    break

Device available now: cpu
(tensor([[-0.3372],
        [ 0.0065],
        [-0.1188],
        [-0.2284],
        [ 0.4132],
        [ 0.0963],
        [-0.0059],
        [-0.1239],
        [ 0.0098],
        [-0.1476],
        [-0.0279],
        [-0.0378],
        [-0.4654],
        [-0.0523],
        [-0.1903],
        [ 0.3583],
        [ 0.2229],
        [ 0.9096],
        [ 0.2470],
        [ 0.1467],
        [-0.4539],
        [ 0.0122],
        [-0.5994],
        [-0.2310],
        [-0.0688],
        [-0.0482],
        [-0.0548],
        [ 0.2929],
        [ 0.0272],
        [-0.0711],
        [ 0.0931],
        [-0.1775],
        [-0.0110],
        [-0.3034],
        [-0.3861],
        [-0.2332],
        [-0.2973],
        [-0.0652],
        [-0.3847],
        [-0.1630],
        [-0.1719],
        [-0.0063],
        [ 0.1056],
        [ 0.0706],
        [-0.3275],
        [-0.2590],
        [-0.4005],
        [ 0.2107],
        [ 0.4316],
        [-0.3753],
        [ 0.0258],
    

# 4. train定義

In [142]:
def evm_score(x_i, x_q, y_i, y_q):
    tmp = 0
    for i in range(len(x_i)):
        tmp += ((x_i[i] - y_i[i]) ** 2 + (x_q[i] - y_q[i]) ** 2) / (y_i[i] ** 2 + y_q[i] ** 2)
    evm = torch.sqrt(tmp / len(x_i))
    return evm

In [155]:
def train_model(device, model, dataloaders_dict, criterion, optimizer, epochs, epochs_section=None):
    for epoch in range(epochs):
        if epochs_section is not None:
            epoch += epochs_section[0]
            end_epoch = epochs_section[1]
        else:
            end_epoch = epochs
        
        start_time = time.time()
        
        for phase in dataloaders_dict.keys():
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            epoch_loss = 0.0
            epoch_evms = 0.0
            
            for x_i, x_q, y_i, y_q in dataloaders_dict[phase]:
                x_i, x_q = x_i.to(device), x_q.to(device)
                y_i, y_q = y_i.to(device), y_q.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    out_i, out_q = model(x_i, x_q)
                    loss = evm_score(out_i, out_q, y_i, y_q)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                    epoch_loss += loss.item() * x_i.size(0)
                    epoch_evms += (evm_score(out_i, out_q, y_i, y_q)) ** 2 * x_i.size(0)
            
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_evm = torch.sqrt(epoch_evms / len(dataloaders_dict[phase].dataset)) * 100
            
            duration = str(datetime.timedelta(seconds=time.time() - start_time))[:7]
            print('{} | Epoch: {}/{} | {} Loss: {:.4} | EVM: {:.4}'.format(duration, epoch + 1, end_epoch, phase, epoch_loss, epoch_evm[0]))
    return model

In [156]:
#動作確認
epochs = 5
lr = 0.001

criterion = nn.MSELoss()
optimizer = optim.Adam(params=model.parameters(), lr=lr)

train_model(device=device, model=model, dataloaders_dict=dataloaders_dict, criterion=criterion, optimizer=optimizer, epochs=epochs);

0:01:00 | Epoch: 1/5 | train Loss: 0.5969 | EVM: 59.72
0:01:02 | Epoch: 2/5 | train Loss: 0.5364 | EVM: 53.69
0:01:14 | Epoch: 3/5 | train Loss: 0.483 | EVM: 48.34
0:01:21 | Epoch: 4/5 | train Loss: 0.4286 | EVM: 42.9


KeyboardInterrupt: 

In [157]:
#結果を保存しない
tap = 201
max_tap = 501
batch_size = 100
hidden_neuron = 300
epochs = 500
lr = 0.001

device = torch.device('cpu') # 'cuda' if torch.cuda.is_available() else 
print('Device available now:', device)

df_dir = '../data/input/'
df0 = pd.read_csv(df_dir+'prbs.csv', index_col=0)

condition0 = (df0['N']==13) & (df0['itr']==1) & (df0['form']=='RZ16QAM') & (df0['n']==32) & (df0['equalize']==False) & (df0['baudrate']==28) & (df0['PdBm']==1)
sgnl0 = load_pickle(df0[condition0].iloc[0]['data_path'])
lc0 = sgnl0.linear_compensation(2500, sgnl0.signal['x_2500'])
x0, y0 = data_shaping(sgnl0.signal['x_0'][16::32], lc0[16::32], max_tap, tap)

condition1 = (df0['N']==17) & (df0['itr']==1) & (df0['form']=='RZ16QAM') & (df0['n']==32) & (df0['equalize']==False) & (df0['baudrate']==28) & (df0['PdBm']==1)
sgnl1 = load_pickle(df0[condition1].iloc[0]['data_path'])
lc1 = sgnl1.linear_compensation(2500, sgnl1.signal['x_2500'])
x1, y1 = data_shaping(sgnl1.signal['x_0'][16::32], lc1[16::32], max_tap, tap)

mean = np.mean(x0)
std = np.std(x0)

train_dataset = Dataset(x=x0, y=y0, mean=mean, std=std)
val_dataset = Dataset(x=x1, y=y1, mean=mean, std=std)

train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
dataloaders_dict = {'train': train_dataloader, 'val': val_dataloader}

model = ComplexANN(input_dim=tap, output_dim=1, hidden_neuron=hidden_neuron).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(params=model.parameters(), lr=lr)

model = train_model(device=device, model=model, dataloaders_dict=dataloaders_dict, criterion=criterion, optimizer=optimizer, epochs=epochs)

Device available now: cpu
0:00:58 | Epoch: 1/500 | train Loss: 1.038 | EVM: 103.8
0:05:59 | Epoch: 1/500 | val Loss: 1.008 | EVM: 100.8
0:01:08 | Epoch: 2/500 | train Loss: 0.7809 | EVM: 78.14


KeyboardInterrupt: 